In [1]:
import sys
import os
import pandas as pd
import pdfplumber
from src.exception import CustomException
from src.ETL_manager import DataIngestor
from src.ETL_manager import DataProcessor
from src.inputs import mapping
from src.inputs.column_structure import get_cols

input_file = '/raw_file/915010018577756.csv'            # Enter you file directory with name
input_file = os.getcwd()+input_file
col_names_1 = get_cols.col1()
df = DataIngestor.read_statement(
    file_path=input_file, 
    col_names=col_names_1
    )


In [2]:
df.head()

,Date,ChqNo,Particulars,Debit,Credit,Balance,BR_Code
0,19-09-2024,-,UPI/P2M/426333754787/Mr S Chandran /UPI...,40.00,,443.64,1355.0
1,19-09-2024,-,UPI/P2M/462916460827/MANIGANDAN /UPI...,20.00,,423.64,1355.0
2,19-09-2024,-,UPI/P2M/426398459712/ASHRAF U K /UPI...,30.00,,393.64,1355.0
3,19-09-2024,-,UPI/P2M/426314447901/HungerBox /Hun...,82.00,,311.64,1355.0
4,19-09-2024,-,UPI/P2M/426336868830/RAMBABU YADAV /UPI...,10.00,,301.64,1355.0


In [3]:
input_file = r'/raw_file/hdfc_statement.xlsx'
input_file = os.getcwd()+input_file
col_names_2 = get_cols.col2()
df_1  = DataIngestor.read_statement(input_file, col_names=col_names_2)

In [4]:
df_new = DataIngestor.clean_features(df, 'Expense')
df_new.head()

,Date,Particulars,Debit,Credit,Balance
0,19-09-2024,UPI/P2M/426333754787/Mr S Chandran /UPI...,40.00,,443.64
1,19-09-2024,UPI/P2M/462916460827/MANIGANDAN /UPI...,20.00,,423.64
2,19-09-2024,UPI/P2M/426398459712/ASHRAF U K /UPI...,30.00,,393.64
3,19-09-2024,UPI/P2M/426314447901/HungerBox /Hun...,82.00,,311.64
4,19-09-2024,UPI/P2M/426336868830/RAMBABU YADAV /UPI...,10.00,,301.64


In [5]:
df_new['Particulars'][3]

'UPI/P2M/426314447901/HungerBox            /Hunger/Yes Bank Ltd'

In [6]:
df_2 = DataProcessor.remap_transactions(df_new, 'Particulars')
df_2.head()

,Date,Particulars,exp_name,Debit,Credit,Balance
0,19-09-2024,Mr S Chandran,None,40.00,,443.64
1,19-09-2024,MANIGANDAN,None,20.00,,423.64
2,19-09-2024,ASHRAF U K,None,30.00,,393.64
3,19-09-2024,HungerBox Hunger,Hunger,82.00,,311.64
4,19-09-2024,RAMBABU YADAV,None,10.00,,301.64


In [7]:
df_2['exp_name'][3]

'Hunger'

In [8]:
df_2.head(15)

,Date,Particulars,exp_name,Debit,Credit,Balance
0,19-09-2024,Mr S Chandran,None,40.00,,443.64
1,19-09-2024,MANIGANDAN,None,20.00,,423.64
2,19-09-2024,ASHRAF U K,None,30.00,,393.64
3,19-09-2024,HungerBox Hunger,Hunger,82.00,,311.64
4,19-09-2024,RAMBABU YADAV,None,10.00,,301.64
5,19-09-2024,Cake way,None,12.00,,289.64
6,19-09-2024,GYANESWAR UPADHAYA,None,9.00,,280.64
7,19-09-2024,Sree Kodandarama Serv,None,210.00,,70.64
8,19-09-2024,sri annapoorneshwary,None,20.00,,50.64
9,19-09-2024,SOURAVMA Remitter,Remitter,,500.00,550.64


In [9]:
df_2.exp_name[1]

In [10]:
mapping_ = mapping.map_subcategory()

In [11]:
df_2 = DataProcessor.map_expense_group(df_2)

/Users/souravm/Documents/Projects/expense_analyzer/src/ETL_manager.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['exp_group'] = data['exp_name'].apply(lambda x:map_defined_expense(x,mapping_dict=mapping_))
/Users/souravm/Documents/Projects/expense_analyzer/src/ETL_manager.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['exp_maps'] = data['Particulars'].apply(lambda x:map_defined_expense(x,mapping_dict=mapping_))
/Users/souravm/Documents/Projects/expense_analyzer/src/ETL_manager.py:16

In [12]:
type(df_2)

pandas.core.frame.DataFrame

In [13]:
df_2.head(15)

,Date,Particulars,exp_name,Debit,Credit,Balance,exp_maps
0,19-09-2024,Mr S Chandran,None,40.00,,443.64,FoodRegular
1,19-09-2024,MANIGANDAN,None,20.00,,423.64,Flower
2,19-09-2024,ASHRAF U K,None,30.00,,393.64,Grocery Regular
3,19-09-2024,HungerBox Hunger,Hunger,82.00,,311.64,Office Food
4,19-09-2024,RAMBABU YADAV,None,10.00,,301.64,Others
5,19-09-2024,Cake way,None,12.00,,289.64,Office Food
6,19-09-2024,GYANESWAR UPADHAYA,None,9.00,,280.64,Tea & Others
7,19-09-2024,Sree Kodandarama Serv,None,210.00,,70.64,Fuel
8,19-09-2024,sri annapoorneshwary,None,20.00,,50.64,Tea & Others
9,19-09-2024,SOURAVMA Remitter,Remitter,,500.00,550.64,Fund Transfer


In [14]:
df_2[df_2['exp_maps'].isna()]

,Date,Particulars,exp_name,Debit,Credit,Balance,exp_maps
19,20-09-2024,Durganand Yadav,None,15.00,,213.64,<NA>
34,22-09-2024,PURUSHOTHAMA N hair c,c,80.00,,864.14,<NA>
36,22-09-2024,RAMKUMARGUPTASOMOOLC Pay to,to,75.00,,765.14,<NA>
37,22-09-2024,Rahul,None,40.00,,725.14,<NA>
55,24-09-2024,Mr BABU N,None,24.00,,290.14,<NA>
